In [2]:
import sys
!{sys.executable} -m pip install awswrangler --index-url https://private.artifactory.foc.zone/api/pypi/rdf-pypi-virtual/simple --extra-index-url https://private.artifactory.foc.zone/artifactory/api/pypi/pypi-remote/simple
!{sys.executable} -m pip install pyathena --index-url https://private.artifactory.foc.zone/api/pypi/rdf-pypi-virtual/simple --extra-index-url https://private.artifactory.foc.zone/artifactory/api/pypi/pypi-remote/simple
# !{sys.executable} -m pip install -r requirements.txt --index-url https://private.artifactory.foc.zone/api/pypi/rdf-pypi-virtual/simple --extra-index-url https://private.artifactory.foc.zone/artifactory/api/pypi/pypi-remote/simple

Looking in indexes: https://private.artifactory.foc.zone/api/pypi/rdf-pypi-virtual/simple, https://private.artifactory.foc.zone/artifactory/api/pypi/pypi-remote/simple
Looking in indexes: https://private.artifactory.foc.zone/api/pypi/rdf-pypi-virtual/simple, https://private.artifactory.foc.zone/artifactory/api/pypi/pypi-remote/simple


In [3]:
sys.path.append(r'../src/') 

In [4]:
import numpy as np
import pandas as pd

import sagemaker
from datetime import datetime

import os
import boto3
import awswrangler as wr

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [5]:
session = boto3.Session()
wr.config.sts_endpoint_url = "https://sts." + session.region_name + ".amazonaws.com"

s3_output_path = "s3://sagemaker-us-east-2-382461412892/athena"

### Arg maps

In [6]:
import inspect
from data import query_registry

args_mapping = {}
required_args = ['start_date', 'end_date', 'table_name', 'limit']

for name, obj in inspect.getmembers(query_registry, lambda x: inspect.isclass(x) and x.__module__ == query_registry.__name__):
    argspec = inspect.getfullargspec(obj)
    args = argspec[0][1:]
    defaults = argspec[3]
    defaults = [None]*(len(args)-len(defaults))+list(defaults)
    args_dict = dict(zip(args, defaults))
    args_mapping[name] = ({k: v for k, v in args_dict.items() if k in required_args}, {k: v for k, v in args_dict.items() if k not in required_args})

In [7]:
args_mapping

{'QueryAccountCreate': ({'start_date': None,
   'end_date': None,
   'table_name': 'accountactivityevent_base',
   'limit': 'ALL'},
  {}),
 'QueryAdobeEvents': ({'start_date': None,
   'end_date': None,
   'table_name': 'adobe_websessionpostevent',
   'limit': 'ALL'},
  {'event_codes': [], 'first_only': False}),
 'QueryAdobeLeadformInput': ({'start_date': None,
   'end_date': None,
   'table_name': 'adobe_dq_prcd_data',
   'limit': 'ALL'},
  {'site_sections': ['ql lander', 'rocket lander'],
   'loan_purpose': '',
   'group_id': ''}),
 'QueryAdobeVisitId': ({'start_date': None,
   'end_date': None,
   'table_name': 'adobe_dq_prcd_data',
   'limit': 'ALL'},
  {}),
 'QueryAdobeVisitInfo': ({'start_date': None,
   'end_date': None,
   'table_name': 'adobe_dq_prcd_data',
   'limit': 'ALL'},
  {'site_sections': [], 'loan_purpose': '', 'group_id': ''}),
 'QueryDataBase': ({'start_date': None,
   'end_date': None,
   'table_name': None,
   'limit': 'ALL'},
  {}),
 'QueryExperimentVisitorData':

In [47]:
map_df = pd.read_excel('Query_db_table_mapping.xlsx')
map_df

,QueryClass,database,table
0,QueryAccountCreate,rm_dp_qtweets_raw_processed_access,accountactivityevent_base
1,QueryAdobeEvents,rktdp_adobe_omniture_raw_processed_access,adobe_websessionpostevent
2,QueryAdobeLeadformInput,rktdp_adobe_omniture_raw_processed_access,adobe_dq_prcd_data
3,QueryAdobeVisitId,rktdp_adobe_omniture_raw_processed_access,adobe_dq_prcd_data
4,QueryAdobeVisitInfo,rktdp_adobe_omniture_raw_processed_access,adobe_dq_prcd_data
5,QueryExperimentVisitorData,rktdp_adobe_omniture_raw_processed_access,adobe_dq_prcd_data
6,QueryLeadSubmission,rm_dp_submissionengine_raw_processed_access,submissionstrings_base
7,QueryLoanIdentifier,rm_northstar_raw_processed_access,loan_identifier_dim
8,QueryMajorMilestone,rm_northstar_raw_processed_access,loan_major_milestone_fact
9,QueryPreloanId,rm_dp_qtweets_raw_processed_access,preloanguidcreateevent_sourcereferenceidentifi...


In [53]:
query_map = {}
for idx, row in map_df.iterrows():
    query_map[row['QueryClass']] = [row['database'], row['table']]

In [54]:
import json
with open("query_db_map.json", "w") as outfile:
    json.dump(query_map, outfile)

In [32]:
import data.make_dataset as make_dataset
output_folder = 's3://sagemaker-us-east-2-534295958235/rcd/queryregistry_unittest/'
args = {'start_date': '2022-06-22', 'end_date': '2022-06-22', 'query_label': 'QueryAdobeEvents', 'database': 'rktdp_adobe_omniture_raw_processed_access', 'event_codes': ['206'], 'table_name': 'adobe_websessionpostevent', 'limit': 'ALL', 'workgroup': 'rcd-datascientist', 'output_filepath': output_folder+'test_one_adobe_event_input.parquet'}

In [34]:
df = make_dataset.main.callback(**{**args})
df

### Get Joined Adobe data with offline milestones

In [6]:
from utils.helper import *

In [8]:
start_date = '2022-07-24'
end_date = '2022-07-27'

In [9]:
# milestone event names
dff, id_bridge = get_joined_dataset(start_date = start_date, end_date= end_date, group_id = '562585:1:0', site_sections = [], event_codes = ['204', '206', '250', '228', '20187', '201'], milestones = ['Lead', 'Allocated', 'Credit', 'PAL', 'VAL'], milestone_long_to_wide_column = 'loanmilestone_eventname', join_lead_created_during_visit_time = True)

Query data for date between 2022-07-24 and 2022-07-27

Get lead ids

No. of Preloan ID records: 19174

No. of Preloan purpose records: 19174

No. of Account create records: 117819

No. of Lead submit records: 243024

No. of joined id records: 368504

Get adobe visit data

No. of event records: 769018

No. of leadform input records: 843

Total no. of joined records by Adobevisitorid: 105

Total no. of joined records by RocketAccountid: 275

Total no. of joined records by LoanNumber: 30

Total no. of joined records by LoanGuid: 248

Total no. of unjoined records with no lead form events: 628

No. of web data records after mapping ids: 843

Get loan milestone data

No. of loanidentifier records: 306541

No. of loan milestone records: 464793

No. of final joined records: 843


In [9]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 843 entries, 0 to 842
Data columns (total 60 columns):
 #   Column                                   Non-Null Count  Dtype                     
---  ------                                   --------------  -----                     
 0   groupidx_string                          843 non-null    object                    
 1   mcvisid_visitorid                        843 non-null    object                    
 2   uniquevisitkey                           843 non-null    object                    
 3   visitnumber                              843 non-null    Int32                     
 4   min_visit_estdatetime                    843 non-null    datetime64[ns, US/Eastern]
 5   max_visit_estdatetime                    843 non-null    datetime64[ns, US/Eastern]
 6   min_landingpage_estdatetime              663 non-null    datetime64[ns, US/Eastern]
 7   leadform_event                           843 non-null    float64                   
 8   

In [56]:
dff.head(1)

,groupidx_string,mcvisid_visitorid,uniquevisitkey,visitnumber,min_visit_estdatetime,max_visit_estdatetime,min_landingpage_estdatetime,leadform_event,min_testexperiment_estdatetime,max_testexperiment_estdatetime,testexperiment,first_testexperiment,first_marketingchannel_finder,first_marketingchannel_closer,first_loanpurpose,last_loanpurpose,first_landing_sitesection,first_homedescription,first_propertyuse,first_creditrating,first_timeframetopurchase,first_firsttimebuyer,first_hasrealestateagent,first_purchaseprice,best_match_loannumber,best_match_loanguid,best_match_rocketaccountid,is_event_201,is_event_20187,is_event_204,is_event_206,is_event_228,is_event_250,201_estdatetime,20187_estdatetime,204_estdatetime,206_estdatetime,228_estdatetime,250_estdatetime,best_match_loanidentifierdimsk,is_milestone_Allocated,is_milestone_Credit Pulled in AMP,is_milestone_Idle Pulled Credit,is_milestone_LOLA Lead Created,is_milestone_PAL,is_milestone_Rocket Credit Pulled,is_milestone_Rocket Lead Created,is_milestone_Rocket Logic Credit Pulled,is_milestone_Salesforce Allocated,is_milestone_VAL,Allocated_estdatetime,Credit Pulled in AMP_estdatetime,Idle Pulled Credit_estdatetime,LOLA Lead Created_estdatetime,PAL_estdatetime,Rocket Credit Pulled_estdatetime,Rocket Lead Created_estdatetime,Rocket Logic Credit Pulled_estdatetime,Salesforce Allocated_estdatetime,VAL_estdatetime
0,"[25, 145]",73100633111118509192862076304428775312,8772439848265554910258549288033333458211658638799,1,2022-07-24 00:59:59-04:00,2022-07-24 01:05:15-04:00,2022-07-24 00:59:59-04:00,1.0,2022-07-24 01:00:42-04:00,2022-07-24 01:05:15-04:00,562585:1:0--**--562432:0:0,562585:1:0,Affiliate Networks,Affiliate Networks,Purchase,Purchase,rocket lander,singlefamily,primaryresidence,<NA>,no time frame; i am researching options,yes,<NA>,<NA>,3510857551,782b7c3c-278c-4553-bdcc-5ec268dfe7ef,36954634-34a7-443b-9a7f-c3534cc5ad9a,NaN,1.0,1.0,NaN,NaN,1.0,NaT,2022-07-24 01:05:15-04:00,2022-07-24 01:01:38-04:00,NaT,NaT,2022-07-24 01:01:44-04:00,230427979.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaT,NaT,2022-07-24 01:05:46-04:00,2022-07-24 01:05:00-04:00,NaT,NaT,2022-07-24 01:01:44-04:00,NaT,NaT,NaT


In [61]:
dff, id_bridge = get_joined_dataset(start_date, end_date, loan_purpose = 'Purchase', event_codes = ['204', '206', '250', '228', '20187'], milestones = ['Allocated', 'Credit', 'PAL', 'VAL'])
# dff, id_bridge = get_joined_dataset(start_date = start_date, end_date= end_date, group_id = '562585:1:0', event_codes = ['204', '206', '250', '228', '20187'], milestones = ['Lead', 'Allocated', 'Credit', 'PAL', 'VAL'])
# dff, id_bridge = get_joined_dataset(start_date = start_date, end_date= end_date, group_id = '562585:1:0', site_sections = [], event_codes = ['204', '206', '250', '228', '20187', '201'], milestones = ['Allocated', 'Credit', 'PAL', 'VAL'])

Query data for date between 2022-07-24 and 2022-07-27

Get lead ids

No. of Preloan ID records: 19174

No. of Preloan purpose records: 13007

No. of Account create records: 117819

No. of Lead submit records: 81483

No. of joined id records: 203283

Get adobe visit data

No. of event records: 97820

No. of leadform input records: 111620

Total no. of joined records by Adobevisitorid: 24906

Total no. of joined records by RocketAccountid: 10871

Total no. of joined records by LoanNumber: 20690

Total no. of joined records by LoanGuid: 9519

Total no. of unjoined records with no lead form events: 82663

No. of web data records after mapping ids: 111647

Get loan milestone data

No. of loanidentifier records: 306538

No. of loan milestone records: 464787

No. of final joined records: 111647


In [62]:
dff.head(1)

,groupidx_string,mcvisid_visitorid,uniquevisitkey,visitnumber,min_visit_estdatetime,max_visit_estdatetime,min_landingpage_estdatetime,leadform_event,min_testexperiment_estdatetime,max_testexperiment_estdatetime,testexperiment,first_testexperiment,first_marketingchannel_finder,first_marketingchannel_closer,first_loanpurpose,last_loanpurpose,first_landing_sitesection,first_homedescription,first_propertyuse,first_creditrating,first_timeframetopurchase,first_firsttimebuyer,first_hasrealestateagent,first_purchaseprice,best_match_loannumber,best_match_loanguid,best_match_rocketaccountid,is_event_20187,is_event_204,is_event_206,is_event_228,is_event_250,20187_estdatetime,204_estdatetime,206_estdatetime,228_estdatetime,250_estdatetime,best_match_loanidentifierdimsk,is_milestone_Allocated,is_milestone_Lead,is_milestone_Offline Credit,is_milestone_Online Credit,is_milestone_PAL,is_milestone_VAL,Allocated_estdatetime,Lead_estdatetime,Offline Credit_estdatetime,Online Credit_estdatetime,PAL_estdatetime,VAL_estdatetime
0,"[97944, 113101]",15453299120763883894560889149729932446,3632607051006903692181468951508705804311658635026,1,2022-07-24 00:00:00-04:00,2022-07-24 00:00:38-04:00,2022-07-24 00:00:00-04:00,1.0,NaT,NaT,560145:0:0,<NA>,Paid Search,Paid Search,Purchase,Purchase,ql lander,singlefamily,primaryresidence,580,6+ months,no,no,<NA>,3510856929,<NA>,<NA>,NaN,NaN,1.0,NaN,NaN,NaT,NaT,2022-07-24 00:00:00-04:00,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT


In [63]:
id_bridge.head(1)

,unique_rocketaccountid,unique_loannumber,unique_loanguid,unique_adobevisitorid
groupidx_string,,,,
"[0, 1, 2]",[a8ef1355-b58d-4b53-a642-4c38123cda01],"[3510874945, 3510874342]",[38837b4d-b3db-4a54-b0ac-24ef59b32b89],[71222555738866517910678967903827244614]


### Query Lead/Account create Data with start and end dates

In [6]:
from data.make_dataset import get_data
from data.query_registry import QueryPreloanPurpose, QueryPreloanId, QueryAccountCreate, QueryLeadSubmission

In [7]:
start_date = '2022-07-18'
end_date = '2022-07-18'

In [36]:
df_preloan_id = get_data(QueryPreloanId(start_date, end_date), "rm_dp_qtweets_raw_processed_access")
print('No. of Preloan ID records:', df_preloan_id.shape[0])

df_preloan_purpose = get_data(QueryPreloanPurpose(start_date, end_date), "rm_dp_qtweets_raw_processed_access")
print('No. of Preloan purpose records:', df_preloan_purpose.shape[0])

df_account = get_data(QueryAccountCreate(start_date, end_date), "rm_dp_qtweets_raw_processed_access")
print('No. of Account create records:', df_account.shape[0])

df_lead = get_data(QueryLeadSubmission(start_date, end_date), "rm_dp_submissionengine_raw_processed_access")
print('No. of Lead submit records:', df_lead.shape[0])

No. of Preloan ID records: 5392
No. of Preloan purpose records: 5392
No. of Account create records: 28837
No. of Lead submit records: 88478


In [18]:
print('df_preloan_id table')
display(df_preloan_id.head(1))
print('\ndf_preloan_purpose table')
display(df_preloan_purpose.head(1))
print('\ndf_account table')
display(df_account.head(1))
print('\ndf_lead table')
display(df_lead.head(1))

df_preloan_id table


,recordid,loanguid,loannumber,rocketaccountid,datekey
0,a441a2442d66943bb1499060a61b4798d3a9ef7a0455e1...,42d8b35c-3c22-40bc-981d-8d6b0db68107,<NA>,e662e6e3-eea7-4a56-95a0-72fef9b48535,2022-07-18



df_preloan_purpose table


,utcdatetime,recordid,loanpurpose,datekey
0,2022-07-18T02:39:14+00:00,dbd4b317dce1e98c64b1c01f56a4f0c67d351da4f02aa8...,Purchase,2022-07-18



df_account table


,utcdatetime,recordid,rocketaccountid,datekey
0,2022-07-18T03:01:22.691Z,6ce7f5c99af05d3a0b4da59041eecf1a7975de43e72d20...,83f2e018-aada-40cc-95e2-044464d70c5a,2022-07-18



df_lead table


,estdatetime,utcdatetime,recordid,leadid,loannumber,adobevisitorid,loanguid,rmclientid,loanidentifier,loanpurpose,leadsystem,leadsource,leadsourcecategory,leadtypecode,typeoflead,loangrade,isduplicate,rlautotransfer,b1personentityid,b1ficoscore,b2personentityid,b2ficoscore,isprimaryresidence,leadgoals,propertyuse,lbtimeframetobuy,selfassessedcredit,hasrealestateagent,fname,mname,lname,email,gcid,gcid2,datekey
0,07/18/2022 23:57:50.967-04:00,2022-07-19T03:57:50.0939034Z,cc5e2078c230660a92f2c93b39640b7de362b4ad218e37...,112650161,3510459868,undefined,<NA>,<NA>,549300FGXN1K3HLB1R50351045986872,Refinance,QLWEB,Quicken Loans,Quicken Loans,CARILIVEQREFI,Website,Grade A,false,<NA>,aae5a57789554e288c86abd827c4f055,<NA>,<NA>,<NA>,false,<NA>,<NA>,<NA>,undefined,<NA>,Sara,<NA>,Mosher,saraelsiemae@gmail.com,182420679,<NA>,2022-07-18


### Transform dataset and join tables

In [90]:
from utils.utilities import *
T = Transformation()
J = Join()

In [91]:
## transform datetimes and filter with Purchase loans
df_preloan = J.join_preloan_id_with_purpose(df_preloan_id, df_preloan_purpose, 'Purchase')
df_account = T.transform_account_df(df_account)
df_lead = T.transform_lead_df(df_lead, 'Purchase')

In [92]:
print('transformed df_preloan table')
display(df_preloan.head(1))
print('\ntransformed df_account table')
display(df_account.head(1))
print('\ntransformed df_lead table')
display(df_lead.head(1))

transformed df_preloan table


,preloan_utcdatetime,preloan_estdatetime,preloan_recordid,loanguid,loannumber,rocketaccountid,loanpurpose
0,2022-07-18 08:56:07+00:00,2022-07-18 04:56:07-04:00,dca2df1f5398be1be09adee0f8f096aea2b59e1a200c47...,a8b0fe0b-9b1f-4f01-b54d-4290216863d1,<NA>,1683f369-9fef-40e8-9535-dc9caeb3af83,Purchase



transformed df_account table


,accountcreate_utcdatetime,accountcreate_estdatetime,accountcreate_recordid,rocketaccountid
0,2022-07-18 22:00:23+00:00,2022-07-18 18:00:23-04:00,ac9a112b79eddbbaa2d031b20e178d67d3e479df2e62e0...,cc7d957f-e4a7-4fe4-82ad-611a513c5b67



transformed df_lead table


,leadsubmit_utcdatetime,leadsubmit_estdatetime,leadsubmit_recordid,loannumber,adobevisitorid,loanguid,rmclientid,loanpurpose,leadsystem,leadsourcecategory,leadtypecode,isduplicate,rlautotransfer
0,2022-07-18 06:38:05+00:00,2022-07-18 02:38:06-04:00,1c44cab1d4bc949a62b637f2b264326f80d049da342126...,3510358234,34782446354988417911590899335241760477,<NA>,<NA>,Purchase,QLWEB,Quicken Loans Exclusive - QL.com,WHALF,false,<NA>


In [93]:
## join ids together
df_id = J.join_lead_with_preloan_with_account(df_lead, df_preloan, df_account)
df_id.shape

(53669, 10)

In [94]:
df_id.head(1)

,leadsubmit_utcdatetime,leadsubmit_estdatetime,loannumber,adobevisitorid,loanguid,rocketaccountid,preloan_utcdatetime,preloan_estdatetime,accountcreate_utcdatetime,accountcreate_estdatetime
0,2022-07-18 10:16:27+00:00,2022-07-18 06:16:28-04:00,3510284215,47591870058482427211796748060302390587,7cda69b2-c3e3-4824-bb8c-bbeebfd5ec11,39bb9f37-fa50-45c7-a493-435811c04c43,2022-07-18 10:04:02+00:00,2022-07-18 06:04:02-04:00,2022-07-18 10:03:57+00:00,2022-07-18 06:03:57-04:00


### Query Adobe Data

In [15]:
from data.query_registry import QueryAdobeEvents, QueryAdobeVisitId, QueryAdobeVisitInfo, QueryAdobeLeadformInput

In [16]:
start_date = '2022-07-18'
end_date = '2022-07-18'

##### - Query Adobe data with specified events

In [17]:
event_codes = ['204', '206', '250']
# event_codes = ['206']
df_event = get_data(QueryAdobeEvents(start_date = start_date, end_date = end_date, event_codes = event_codes), "rktdp_adobe_omniture_raw_processed_access")
df_event.shape

(20014, 4)

In [18]:
df_event.head(1)

,date_time,uniquevisitkey,mcvisid_visitorid,eventid
0,2022-07-18 08:16:50,1073863628764871830622715696950275984171658146610,71716007344118836740113846517902348830,206


In [19]:
df_event['eventid'].value_counts()

206    11170
250     5179
204     3665
Name: eventid, dtype: Int64

##### - Query Adobe data for available ids at visit level

In [53]:
df_visit_id = get_data(QueryAdobeVisitId(start_date, end_date), "rktdp_adobe_omniture_raw_processed_access")
df_visit_id.shape

(757256, 7)

In [54]:
df_visit_id.head(1)

,min_visit_date_time,uniquevisitkey,mcvisid_visitorid,rocketaccountid_evar5,rocketmortgage_loannumber_evar8,rocketmortgage_loanguid_evar115,datekey
0,2022-07-18 05:22:47,1421516316977674293364486109528442555921658136167,32758823974756695373411929395461818212,7a53e412-88ff-422f-986a-514e9e7bffd2,3312016749,<NA>,2022-07-18


##### - Query Adobe data for visit level information

In [20]:
df_visit_info = get_data(QueryAdobeVisitInfo(start_date, end_date), "rktdp_adobe_omniture_raw_processed_access")
df_visit_info.shape

(757256, 22)

In [21]:
df_visit_info.head(1)

,uniquevisitkey,mcvisid_visitorid,visitnumber,min_visit_date_time,max_visit_date_time,rocketaccountid_evar5,rocketmortgage_loannumber_evar8,rocketmortgage_loanguid_evar115,first_sitesection,visit_ql_lander,visit_rocket_lander,first_loanpurpose,last_loanpurpose,first_pagename,first_qls,first_va_finder_id,first_va_closer_id,first_referrer,first_geo_city,first_geo_zip,first_carrier,first_domain
0,1000064811737223776546714191339886410911658173098,46115048819831788321061145202856503741,1,2022-07-18 15:38:18,2022-07-18 15:46:41,<NA>,3510429294,<NA>,ql lander,1,0,Purchase,Purchase,ql:lander:wham:loanType,GBM_SL7ARMLN.rates0000e,1,1,https://www.google.com/,los angeles,90006,myvzw.com:verizon business,myvzw.com


##### - Query Adobe data for RM leadform inputs (this applies to users who visited ql landing page or rocket landing page)

In [10]:
site_sections = ['ql lander']
df_leadform_input = get_data(QueryAdobeLeadformInput(start_date, end_date, site_sections = site_sections), "rktdp_adobe_omniture_raw_processed_access")
df_leadform_input.shape

(97045, 35)

In [12]:
df_leadform_input.head(1)

,uniquevisitkey,mcvisid_visitorid,visitnumber,min_visit_date_time,max_visit_date_time,rocketaccountid_evar5,rocketmortgage_loannumber_evar8,rocketmortgage_loanguid_evar115,min_rocketaccountid_date_time,min_loannumber_date_time,min_loanguid_date_time,first_loanpurpose,last_loanpurpose,first_landing_sitesection,min_landingpage_date_time,first_homedescription,last_homedescription,first_propertyuse,last_propertyuse,first_creditrating,last_creditrating,first_timeframetopurchase,last_timeframetopurchase,first_firsttimebuyer,last_firsttimebuyer,first_hasrealestateagent,last_hasrealestateagent,first_purchaseprice,last_purchaseprice,first_testexperiment,testexperiment,min_testexperiment_date_time,max_testexperiment_date_time,first_va_finder_id,first_va_closer_id
0,1011415331412399449266461792738609043111658153201,39800228008088986229182245199822772439,1,2022-07-18 10:06:41,2022-07-18 10:07:54,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Purchase,Purchase,ql lander,2022-07-18 10:06:41,condominium,condominium,primaryresidence,primaryresidence,620,620,2 - 3 months,2 - 3 months,yes,yes,no,no,"113,051","113,051",<NA>,<NA>,<NA>,<NA>,5,5


### Transform Adobe Data

In [36]:
## transform event table from long to wide
df_event = T.transform_event_df(df_event)

In [37]:
df_event.head(1)

,uniquevisitkey,mcvisid_visitorid,is_event_204,204_estdatetime,204_utcdatetime,is_event_206,206_estdatetime,206_utcdatetime,is_event_250,250_estdatetime,250_utcdatetime,leadform_event
0,1000055912008769367364306512509662790911658197996,46825191632046187613012207811286996766,NaN,NaT,NaT,NaN,NaT,NaT,1.0,2022-07-18 22:33:16-04:00,2022-07-19 02:33:16+00:00,1.0


In [38]:
## transform leadform input table, convert datetimes to right data type
df_leadform_input = T.transform_leadform_input_df(df_leadform_input)
df_leadform_input.head(1)

,uniquevisitkey,mcvisid_visitorid,visitnumber,rocketaccountid_evar5,rocketmortgage_loannumber_evar8,rocketmortgage_loanguid_evar115,first_loanpurpose,last_loanpurpose,first_landing_sitesection,first_homedescription,last_homedescription,first_propertyuse,last_propertyuse,first_creditrating,last_creditrating,first_timeframetopurchase,last_timeframetopurchase,first_firsttimebuyer,last_firsttimebuyer,first_hasrealestateagent,last_hasrealestateagent,first_purchaseprice,last_purchaseprice,first_testexperiment,testexperiment,first_va_finder_id,first_va_closer_id,first_marketingchannel_finder,first_marketingchannel_closer,min_visit_estdatetime,max_visit_estdatetime,min_landingpage_estdatetime,min_testexperiment_estdatetime,max_testexperiment_estdatetime,min_rocketaccountid_estdatetime,min_loannumber_estdatetime,min_loanguid_estdatetime
0,100624697080559429843622045201324625111658182455,43604395740635473880987982284046349324,1,<NA>,<NA>,<NA>,Purchase,Purchase,ql lander,singlefamily,singlefamily,primaryresidence,primaryresidence,<NA>,<NA>,6+ months,6+ months,no,no,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,1,Paid Search,Paid Search,2022-07-18 18:14:15-04:00,2022-07-18 18:15:47-04:00,2022-07-18 18:14:15-04:00,NaT,NaT,NaT,NaT,NaT


### Join web lead form inputs with events at visit level

In [34]:
## filter with Purchase transactions
df_lead_visit_event = J.join_event_visitor_with_leadform_input(df_event, df_leadform_input, 'Purchase')
df_lead_visit_event.shape

(29999, 47)

In [35]:
df_lead_visit_event['leadform_event'].value_counts()

0.0    22349
1.0     7650
Name: leadform_event, dtype: int64

In [36]:
df_lead_visit_event.head(1)

,uniquevisitkey,mcvisid_visitorid,visitnumber,rocketaccountid_evar5,rocketmortgage_loannumber_evar8,rocketmortgage_loanguid_evar115,first_loanpurpose,last_loanpurpose,first_landing_sitesection,first_homedescription,last_homedescription,first_propertyuse,last_propertyuse,first_creditrating,last_creditrating,first_timeframetopurchase,last_timeframetopurchase,first_firsttimebuyer,last_firsttimebuyer,first_hasrealestateagent,last_hasrealestateagent,first_purchaseprice,last_purchaseprice,first_testexperiment,testexperiment,first_va_finder_id,first_va_closer_id,first_marketingchannel_finder,first_marketingchannel_closer,min_visit_estdatetime,max_visit_estdatetime,min_landingpage_estdatetime,min_testexperiment_estdatetime,max_testexperiment_estdatetime,min_rocketaccountid_estdatetime,min_loannumber_estdatetime,min_loanguid_estdatetime,is_event_204,204_estdatetime,204_utcdatetime,is_event_206,206_estdatetime,206_utcdatetime,is_event_250,250_estdatetime,250_utcdatetime,leadform_event
0,1079876203313143651501579845766219723311658191188,91258645912290785273342754133513831585,1,<NA>,<NA>,<NA>,Purchase,Purchase,ql lander,singlefamily,singlefamily,primaryresidence,primaryresidence,660,660,within 30 days,within 30 days,yes,yes,no,no,"300,000","300,000",565001:3:0,565001:3:0,1,1,Paid Search,Paid Search,2022-07-18 20:39:48-04:00,2022-07-18 20:41:52-04:00,2022-07-18 20:39:48-04:00,2022-07-18 20:39:48-04:00,2022-07-18 20:41:52-04:00,NaT,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,0.0


### Join lead ids with web leads data

In [37]:
## this function joins ids with loannumber, loanguid, rocketaccountid and adobevisitorid. 
## Records in df_lead_visit_event are uniquely identified by uniquevisitkey
dff = J.join_lead_id_with_web_leads(df_id, df_lead_visit_event, unique_id = 'uniquevisitkey')


Total no. of joined records by Adobevisitorid: 6762

Total no. of joined records by RocketAccountid: 2342

Total no. of joined records by LoanNumber: 5740

Total no. of joined records by LoanGuid: 2287

Total no. of unjoined records with no lead form events: 22201


In [38]:
## map ids that associate to one lead/loan. get id bridge table
dff, ID_bridge = get_id_bridge_in_web_leads_data(dff)

In [39]:
ID_bridge.head(2)

,unique_rocketaccountid,unique_loannumber,unique_loanguid,unique_adobevisitorid
groupidx_string,,,,
"[0, 1]",[75a3e9ed-072a-47b3-bca9-ba1323e48293],[3510414249],[09c985f7-90f4-42aa-b4f5-9f79b0401acb],[32189542511065189444344971190874731891]
"[10, 6773, 6774]",[828bc854-ca80-42e7-9d89-29fca29fc029],[3510455686],[bcfd057b-7b91-447e-bf65-b760887dd73c],[57183876964969537922659994473066133644]


### Get cleaned join records

In [40]:
df_web_data = get_matched_web_leads_data(dff)
df_web_data.shape

(29999, 33)

In [41]:
df_web_data.head(2)

,groupidx_string,mcvisid_visitorid,uniquevisitkey,visitnumber,min_visit_estdatetime,max_visit_estdatetime,min_landingpage_estdatetime,leadform_event,min_testexperiment_estdatetime,max_testexperiment_estdatetime,testexperiment,first_testexperiment,first_marketingchannel_finder,first_marketingchannel_closer,first_loanpurpose,last_loanpurpose,first_landing_sitesection,first_homedescription,first_propertyuse,first_creditrating,first_timeframetopurchase,first_firsttimebuyer,first_hasrealestateagent,first_purchaseprice,best_match_loannumber,best_match_loanguid,best_match_rocketaccountid,is_event_204,is_event_206,is_event_250,204_estdatetime,206_estdatetime,250_estdatetime
0,"[0, 1]",32189542511065189444344971190874731891,38558430850918809185243612801779692311658166918,1,2022-07-18 13:55:18-04:00,2022-07-18 14:50:19-04:00,2022-07-18 13:55:18-04:00,1.0,2022-07-18 14:16:10-04:00,2022-07-18 14:50:19-04:00,563956:0:0--**--560145:0:0--**--562432:1:0,562432:1:0,Paid Search,Paid Search,Purchase,Purchase,ql lander,singlefamily,primaryresidence,720+,no time frame; i am researching options,yes,no,"200,000",3510414249,09c985f7-90f4-42aa-b4f5-9f79b0401acb,75a3e9ed-072a-47b3-bca9-ba1323e48293,1.0,1.0,1.0,2022-07-18 14:16:01-04:00,2022-07-18 13:57:45-04:00,2022-07-18 14:16:10-04:00
1,"[10, 6773, 6774]",57183876964969537922659994473066133644,2291748875663444536395439704051323338521658176877,2,2022-07-18 16:41:17-04:00,2022-07-18 16:58:23-04:00,2022-07-18 16:41:17-04:00,1.0,2022-07-18 16:43:09-04:00,2022-07-18 16:58:23-04:00,559718:0:0--**--562032:0:0--**--556112:0:0--**...,562432:1:0,Natural Search,Paid Search,Purchase,Purchase,rocket lander,singlefamily,primaryresidence,720+,no time frame; i am researching options,yes,<NA>,<NA>,3510455686,bcfd057b-7b91-447e-bf65-b760887dd73c,828bc854-ca80-42e7-9d89-29fca29fc029,1.0,NaN,1.0,2022-07-18 16:42:40-04:00,NaT,2022-07-18 16:43:09-04:00


### Query Loan Milestone Data

In [32]:
from data.query_registry import QueryMajorMilestone, QueryLoanIdentifier

In [19]:
start_date = '2022-07-18'
end_date = '2022-07-18'

In [20]:
df_loanidentifier = get_data(QueryLoanIdentifier(start_date, end_date), "rm_northstar_raw_processed_access")
print('\nNo. of loanidentifier records:', df_loanidentifier.shape[0])

df_milestone = get_data(QueryMajorMilestone(start_date = start_date, end_date = end_date, milestones=[]), "rm_northstar_raw_processed_access")
print('\nNo. of loan milestone records:', df_milestone.shape[0])


No. of loanidentifier records: 107637

No. of loan milestone records: 109398


In [67]:
print('df_loanidentifier table')
display(df_loanidentifier.head(1))
print('\ndf_milestone table')
display(df_milestone.head(1))

df_loanidentifier table


,loanidentifierdimsk,loannumber,loanguid
0,229956899,3510396429,<NA>



df_milestone table


,loanidentifierdimsk,loanmilestonesk,loanmilestone_eventname,loanmilestone_groupname,estdatetime
0,229968844,7,LOLA Lead Created,Lead,2022-07-18 12:01:00


### Transform dataset and join milestones with loan identifiers

In [46]:
df_loanidentifier = T.transform_loanidentifier_df(df_loanidentifier)
df_milestone = T.transform_milestone_df(df_milestone, long_to_wide = True, long_to_wide_column = 'loanmilestone_groupname')

In [47]:
print('transformed df_loanidentifier table')
display(df_loanidentifier.head(1))
print('\ntransformed df_milestone table')
display(df_milestone.head(1))

transformed df_loanidentifier table


,loanidentifierdimsk,loannumber,loanguid
0,229919863,3510357825,002799dd-40ce-4654-a7bd-e493dda7724e



transformed df_milestone table


,loanidentifierdimsk,min_Lead_estdatetime,is_milestone_Allocated,Allocated_estdatetime,Allocated_utcdatetime,is_milestone_Application,Application_estdatetime,Application_utcdatetime,is_milestone_Closing,Closing_estdatetime,Closing_utcdatetime,is_milestone_Folder,Folder_estdatetime,Folder_utcdatetime,is_milestone_Lead,Lead_estdatetime,Lead_utcdatetime,is_milestone_Net Leads,Net Leads_estdatetime,Net Leads_utcdatetime,is_milestone_Offline Credit,Offline Credit_estdatetime,Offline Credit_utcdatetime,is_milestone_Online Credit,Online Credit_estdatetime,Online Credit_utcdatetime,is_milestone_PAL,PAL_estdatetime,PAL_utcdatetime,is_milestone_VAL,VAL_estdatetime,VAL_utcdatetime
0,1373063,NaT,1.0,2022-07-18 21:34:48-04:00,2022-07-19 01:34:48+00:00,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT


In [48]:
## joins with loan identifiers
df_loanidentifier_milestone = J.join_loanidentifier_with_milestone(df_loanidentifier, df_milestone)

In [49]:
df_loanidentifier_milestone.head(1)

,loanidentifierdimsk,loannumber,loanguid,min_Lead_estdatetime,is_milestone_Allocated,Allocated_estdatetime,Allocated_utcdatetime,is_milestone_Application,Application_estdatetime,Application_utcdatetime,is_milestone_Closing,Closing_estdatetime,Closing_utcdatetime,is_milestone_Folder,Folder_estdatetime,Folder_utcdatetime,is_milestone_Lead,Lead_estdatetime,Lead_utcdatetime,is_milestone_Net Leads,Net Leads_estdatetime,Net Leads_utcdatetime,is_milestone_Offline Credit,Offline Credit_estdatetime,Offline Credit_utcdatetime,is_milestone_Online Credit,Online Credit_estdatetime,Online Credit_utcdatetime,is_milestone_PAL,PAL_estdatetime,PAL_utcdatetime,is_milestone_VAL,VAL_estdatetime,VAL_utcdatetime
0,229919863,3510357825,002799dd-40ce-4654-a7bd-e493dda7724e,2022-07-18 01:21:10-04:00,1.0,2022-07-18 08:01:40-04:00,2022-07-18 12:01:40+00:00,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,1.0,2022-07-18 01:21:10-04:00,2022-07-18 05:21:10+00:00,1.0,2022-07-18 01:21:10-04:00,2022-07-18 05:21:10+00:00,1.0,2022-07-18 01:40:11-04:00,2022-07-18 05:40:11+00:00,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT


### Get final output records

In [50]:
dff_output = J.join_loanstatus_with_web_leads_data(df_web_data, df_loanidentifier_milestone)   

In [51]:
dff_output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29999 entries, 0 to 29998
Data columns (total 54 columns):
 #   Column                          Non-Null Count  Dtype                     
---  ------                          --------------  -----                     
 0   groupidx_string                 29999 non-null  object                    
 1   mcvisid_visitorid               29999 non-null  object                    
 2   uniquevisitkey                  29999 non-null  object                    
 3   visitnumber                     29999 non-null  Int32                     
 4   min_visit_estdatetime           29999 non-null  datetime64[ns, US/Eastern]
 5   max_visit_estdatetime           29999 non-null  datetime64[ns, US/Eastern]
 6   min_landingpage_estdatetime     29999 non-null  datetime64[ns, US/Eastern]
 7   leadform_event                  29999 non-null  float64                   
 8   min_testexperiment_estdatetime  13485 non-null  datetime64[ns, US/Eastern]
 9   max_te